In [3]:
import pandas as pd
from datetime import datetime, timedelta
import os
from tqdm import tqdm
import pickle
tqdm.pandas()
import warnings
import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
warnings.filterwarnings("ignore")

### One file

In [7]:
df_train = pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/phase_1/00000_to_02284-initial_states.csv')
df_train = pd.concat([df_train,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/phase_1/02285_to_02357-initial_states.csv')])
df_train = pd.concat([df_train,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/phase_1/02358_to_04264-initial_states.csv')])
df_train = pd.concat([df_train,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/phase_1/04265_to_05570-initial_states.csv')])
df_train = pd.concat([df_train,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/phase_1/05571_to_05614-initial_states.csv')])
df_train = pd.concat([df_train,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/phase_1/05615_to_06671-initial_states.csv')])
#df_train = pd.concat([df_train,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/phase_1/06672_to_08118-initial_states.csv')])

df_train['Timestamp'] = pd.to_datetime(df_train['Timestamp'])
df_train = df_train.set_index('File ID')

In [3]:
with open('data_isaac/omni.pickle', 'rb') as f:
    omni = pickle.load(f)
with open('data_isaac/sat.pickle', 'rb') as f:
    sat = pickle.load(f)

In [ ]:
# all_files_omni = os.listdir("phase_1/omni2/")
# all_files_dens = os.listdir("phase_1/sat_density/")
# omni = {}
# for i in tqdm(all_files_omni):
#     omni[int(i.split('-')[1])] = pd.read_csv('phase_1/omni2/'+i)
# sat = {}
# for i in tqdm(all_files_dens):
#     sat[int(i.replace('gr-of','grof').split('-')[1])] = pd.read_csv('phase_1/sat_density/'+i)
# with open('omni.pickle', 'wb') as handle:
#     pickle.dump(omni, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('sat.pickle', 'wb') as handle:
#     pickle.dump(sat, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Create feature by files

In [ ]:
def create_features_and_target(id,df_train):
    all_files_omni = os.listdir("phase_1/omni2/")
    all_files_dens = os.listdir("phase_1/sat_density/")
    file_omni = [i for i in all_files_omni if i.split('-')[1]==id][0]
    file_sat = [i for i in all_files_dens if i.split('-')[1]==id][0]
    data_omni = pd.read_csv('phase_1/omni2/'+file_omni)
    #data_omni['Timestamp'] = pd.to_datetime(data_omni['Timestamp'])
    data_omni = data_omni.ffill()

    data_sat = pd.read_csv('phase_1/sat_density/'+file_sat)
    #data_sat['Timestamp'] = pd.to_datetime(data_sat['Timestamp'])

    df_features = pd.concat([data_omni.drop('Timestamp',axis=1).mean(0),data_omni.iloc[-1]],axis=1)
    df_features.columns = ['mean','last']
    df_features = df_features.unstack()
    df_features.index = df_features.index.get_level_values(0)+'_'+ df_features.index.get_level_values(1)
    print(df_features)
    df_features.loc['Target'] = data_sat['Orbit Mean Density (kg/m^3)'].mean()

    df_position = df_train.loc[df_train['File ID']==int(id)].iloc[0]
    df_features = pd.concat([df_position,df_features],axis=0)
    return df_features

### Create feature by dict

In [ ]:
def create_features_and_target_by_dict(id,df_train,omni,sat,predict_mean=True):
    
    data_omni = omni[id]
    #data_omni['Timestamp'] = pd.to_datetime(data_omni['Timestamp'])
    data_omni = data_omni.ffill()

    data_sat = sat[id]
    data_sat['Timestamp'] = pd.to_datetime(data_sat['Timestamp'],format='%Y-%m-%d %H:%M:%S')

    df_features = pd.concat([data_omni.drop('Timestamp',axis=1).mean(0),data_omni.iloc[-1]],axis=1)
    df_features.columns = ['mean','last']
    df_features = df_features.unstack()
    df_features.index = df_features.index.get_level_values(0)+'_'+ df_features.index.get_level_values(1)
    df_features['last_Timestamp'] = pd.to_datetime(df_features['last_Timestamp'],format='%Y-%m-%d %H:%M:%S')
    df_position = df_train.loc[id]
    df_features = pd.concat([df_position,df_features],axis=0)
    if not predict_mean :
        df_features = pd.concat([df_features.to_frame().T,data_sat]).ffill().dropna(subset='Orbit Mean Density (kg/m^3)')
        df_features['Time_until_y'] = (df_features['Timestamp'] - df_features['last_Timestamp']).dt.seconds.copy()
    else:
        df_features.loc['Target'] = data_sat['Orbit Mean Density (kg/m^3)'].mean()

    return df_features

In [ ]:
model = cb.CatBoostRegressor()
mode.

In [ ]:
def create_pred_timestamps(
    
    tm):
    new_tm = tm + timedelta(seconds=10-tm.second%10)
    date_range = pd.date_range(start=new_tm,periods=432,freq='10min')
    return date_range

In [ ]:
from utils import create_features_for_prediction

In [ ]:
create_features_for_prediction(1,df_train,'dataset/test/omni/omni2-00001-20000604_to_20000803.csv')

In [ ]:
sat[101]

In [ ]:
res = df_train.reset_index()['File ID'].progress_apply(lambda win:create_features_and_target_by_dict(win,df_train,omni,sat))

### Prediction by mean

In [ ]:
model_mean = cb.CatBoostRegressor(loss_function='RMSE',iterations=10000)
model_mean.fit(res.drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp','Target'],axis=1).iloc[:6000],res['Target'].iloc[:6000])

In [ ]:
model_mean.score(res.drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp','Target'],axis=1).iloc[:6000],res['Target'].iloc[:6000])

In [ ]:
model_mean.score(res.drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp','Target'],axis=1).iloc[6000:],res['Target'].iloc[6000:])

In [ ]:
plt.scatter(model_mean.predict(res.drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp','Target'],axis=1).iloc[6000:]),res['Target'].iloc[6000:])

### Prediction by timestamp to predict

In [ ]:
df_training_set = pd.concat(res.iloc[:4000].values)
df_test_set = pd.concat(res.iloc[4000:].values)
df_training_set = df_training_set.drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp'],axis=1)
df_test_set = df_test_set.drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp'],axis=1)
# df_training_set.to_pickle('data_isaac/df_training_set.pkl')
# df_test_set.to_pickle('data_isaac/df_test_set.pkl')
train_dataset = cb.Pool(df_training_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_training_set['Orbit Mean Density (kg/m^3)']) 
test_dataset = cb.Pool(df_test_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_test_set['Orbit Mean Density (kg/m^3)'])

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE',iterations=20000)
model.fit(train_dataset)

In [ ]:
model.save_model('data_isaac/model_by_timestamp.cbm')

In [ ]:
data = pd.DataFrame({'feature_importance': model.feature_importances_, 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)

In [ ]:
id = 6090
res[id]['Orbit Mean Density (kg/m^3)'].plot()
plt.twinx()
plt.plot(model.predict(res[id]),color='red')

In [ ]:
model.score(test_dataset)


###  Compare pred

In [ ]:
id = 90


In [ ]:
df_pred_by_mean = pd.DataFrame(create_pred_timestamps(df_train['Timestamp'].loc[id]),columns=['Timestamp'])
df_pred_by_mean['Orbit Mean Density (kg/m^3)'] = model_mean.predict(create_features_and_target_by_dict(id,df_train,omni,sat).drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp','Target']))

In [ ]:
pred_feature = create_features_and_target_by_dict(id,df_train,omni,sat,predict_mean=False)
df_pred = pd.DataFrame(model.predict(pred_feature.drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp'],axis=1)),
             index=pred_feature['Timestamp'],columns=['Orbit Mean Density (kg/m^3)']).reset_index()

In [ ]:
df_pred_msis = pd.read_csv('data_isaac/sat_density_pred/density_pred'+str(id)+ '.csv',index_col=0)
df_pred_msis['Timestamp'] = pd.to_datetime(df_pred_msis['Timestamp'])

In [ ]:
df_pred_msis.shape

In [ ]:
df_true = sat[id]

In [ ]:
plt.figure()
df_true.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
df_pred.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
df_pred_by_mean.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
df_pred_msis.set_index('Timestamp')['Density (kg/m3)'].plot()

In [ ]:
df_pred_msis.set_index('Timestamp')['Density (kg/m3)'].plot()


###  Model by MSIS

In [4]:
from atm_me import PersistenceMSIS, MSISPersistenceAtmosphere, PersistenceModel

In [5]:
bad_id = []

In [8]:
model = PersistenceModel(plot_trajectory=False)
for id in df_train.index:
    if os.path.exists('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv'):
        continue
    else:
        bad_id.append(id)
        continue
    omni_data = omni[id].loc[:,['Timestamp', 'f10.7_index', 'ap_index_nT']]
    omni_data['Timestamp'] = pd.to_datetime(omni_data['Timestamp'])
    omni_data = omni_data.ffill()
    try:
        states, densities = model(omni_data,df_train.loc[id].to_dict())
        predictions = model._convert_to_df(states, densities)
        predictions.to_csv('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv')
    except:
        print(id)

In [34]:
from poliastro.core.elements import coe2rv

In [35]:
from poliastro.constants.general import GM_earth

In [39]:
coe2rv(GM_earth,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270)

array([[   2040.17287183,    4863.39987966,   -4345.71876999],
       [-223023.87236553,   92295.22958104,   -2829.62704249]])

In [37]:
coe2rv(GM_earth,6824.556715,0.004397,87.269194,127.738000,86.817000,273.818611)

array([[-1.42913713e+02, -6.31702458e+03, -2.64780460e+03],
       [ 1.99207842e+05, -5.56003798e+04,  1.23215306e+05]])

In [38]:
df_train.loc[df_train['Latitude (deg)']<100]

,Timestamp,Semi-major Axis (km),Eccentricity,Inclination (deg),RAAN (deg),Argument of Perigee (deg),True Anomaly (deg),Latitude (deg),Longitude (deg),Altitude (km)
File ID,,,,,,,,,,
0,2000-08-02 04:50:33,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270,43.637815,-62.543128,466.448890
1,2000-08-03 19:51:01,6826.327748,0.003879,87.275694,143.529694,250.438806,109.273118,43.444458,70.139709,463.435053
2,2000-08-05 05:40:05,6819.634802,0.004114,87.268611,142.972111,244.549389,115.138737,19.764250,104.521278,471.625453
3,2000-08-06 05:02:20,6819.606603,0.004134,87.268194,142.608389,241.172000,118.545161,12.450738,112.239558,470.385914
4,2000-08-08 20:54:57,6819.425918,0.004178,87.264611,141.605111,228.779611,130.982981,-8.776992,-130.559634,468.911226
...,...,...,...,...,...,...,...,...,...,...
6667,2016-08-24 12:11:50,6733.514992,0.000718,88.998889,4.216194,149.985500,210.182473,-72.497256,31.068689,383.681828
6668,2016-08-26 20:45:19,6733.346035,0.000752,88.998583,3.877194,142.527806,217.651193,-72.807950,-99.898727,384.260082
6669,2016-08-27 21:12:44,6733.280764,0.000767,88.998694,3.730611,139.466194,220.717390,-72.579717,-107.929470,384.287838


In [29]:
df_train.loc[bad_id]

,Timestamp,Semi-major Axis (km),Eccentricity,Inclination (deg),RAAN (deg),Argument of Perigee (deg),True Anomaly (deg),Latitude (deg),Longitude (deg),Altitude (km)
File ID,,,,,,,,,,
492,2002-10-08 17:10:24,6776.994645,0.001696,87.262194,201.738417,24.771306,335.438282,-18.207303,107.506300,417.278102
569,2003-02-01 02:42:05,6782.012039,0.000303,87.257694,157.393194,29.944000,330.208976,-11.747327,166.175802,411.194035
570,2003-02-03 11:52:14,6781.869755,0.000258,87.257111,156.479500,29.534000,330.604053,-11.987092,25.386467,410.911805
571,2003-02-04 15:40:55,6781.771723,0.000261,87.256889,156.034694,22.815194,337.322604,-12.049309,-33.364383,410.471388
574,2003-02-09 02:17:15,6781.472080,0.000172,87.257500,154.330083,19.380889,340.751776,-12.404173,161.486378,410.012464
...,...,...,...,...,...,...,...,...,...,...
6667,2016-08-24 12:11:50,6733.514992,0.000718,88.998889,4.216194,149.985500,210.182473,-72.497256,31.068689,383.681828
6668,2016-08-26 20:45:19,6733.346035,0.000752,88.998583,3.877194,142.527806,217.651193,-72.807950,-99.898727,384.260082
6669,2016-08-27 21:12:44,6733.280764,0.000767,88.998694,3.730611,139.466194,220.717390,-72.579717,-107.929470,384.287838


In [28]:
df_train.loc[df_train.index.isin(bad_id)==False]

,Timestamp,Semi-major Axis (km),Eccentricity,Inclination (deg),RAAN (deg),Argument of Perigee (deg),True Anomaly (deg),Latitude (deg),Longitude (deg),Altitude (km)
File ID,,,,,,,,,,
0,2000-08-02 04:50:33,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270,43.637815,-62.543128,466.448890
1,2000-08-03 19:51:01,6826.327748,0.003879,87.275694,143.529694,250.438806,109.273118,43.444458,70.139709,463.435053
2,2000-08-05 05:40:05,6819.634802,0.004114,87.268611,142.972111,244.549389,115.138737,19.764250,104.521278,471.625453
3,2000-08-06 05:02:20,6819.606603,0.004134,87.268194,142.608389,241.172000,118.545161,12.450738,112.239558,470.385914
4,2000-08-08 20:54:57,6819.425918,0.004178,87.264611,141.605111,228.779611,130.982981,-8.776992,-130.559634,468.911226
...,...,...,...,...,...,...,...,...,...,...
6160,2013-07-05 11:55:15,6814.005672,0.001713,89.020389,164.408389,95.074083,52.342715,-60.222029,60.089380,469.367913
6161,2013-07-06 23:19:29,6813.956355,0.001709,89.020611,164.209806,90.387306,327.705782,-31.031105,69.692390,454.551635
6162,2013-07-09 10:50:12,6813.873715,0.001697,89.021389,163.875806,82.373500,64.065666,-61.478257,71.861085,468.410940


In [10]:
len(bad_id)


710

In [ ]:
from evaluation_me import DensityModelEvaluator

